## Singapore Private Property 
This set of python codes serve to download past 5 years' of URA private property transactions from URA API for purpose of providing a basis for further data analysis.

The raw data will undergo basic cleaning and then saved into csv file for archival (because URA will continuously remove data older than 5 years old).

In [1]:
import pandas as pd
import requests
from collections import Counter
import datetime as dt
import re

In [2]:
#token given by URA
ura_access_key = 'd8722f05-25ff-44f6-bb6f-5d728aa4c9b1'

#getting the token for the day
r = requests.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey":ura_access_key})
token = r.json()['Result']

In [3]:
#accessing the data in 4 batches as required by URA API
for i in range(1,5):
    #requesting data by passing in access key and token, in 4 batches as stated in URA API website
    req = requests.get('https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=' + str(i),
                      headers={"AccessKey":ura_access_key, "Token":token})
    
    df_temp = pd.json_normalize(req.json()['Result'], 'transaction', ['street','x','y','project'],errors='ignore')

    #adding each batch to DataFrame
    if i==1:
        df = df_temp
    else:
        df = df.append(df_temp, ignore_index=True) #ignore index so that append can be done
    
    #printing the progress for monitoring
    print('Batch ' + str(i) + ' completed')

Batch 1 completed
Batch 2 completed
Batch 3 completed
Batch 4 completed


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113032 entries, 0 to 113031
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   area          113032 non-null  object
 1   floorRange    113032 non-null  object
 2   noOfUnits     113032 non-null  object
 3   contractDate  113032 non-null  object
 4   typeOfSale    113032 non-null  object
 5   price         113032 non-null  object
 6   propertyType  113032 non-null  object
 7   district      113032 non-null  object
 8   typeOfArea    113032 non-null  object
 9   tenure        113032 non-null  object
 10  nettPrice     1137 non-null    object
 11  street        113032 non-null  object
 12  x             88177 non-null   object
 13  y             88177 non-null   object
 14  project       113032 non-null  object
dtypes: object(15)
memory usage: 12.9+ MB


In [23]:
#saving raw data to local drive
this_year = dt.date.today().year
this_month = dt.date.today().month

df.to_csv("ura_caveats downloaded year " + str(this_year) + ' month ' + str(this_month) + " raw data.csv")

In [24]:
#making copy of df
df1 = df.copy()

#performing basic tidying up

#nett price is final selling price, hence will replace price if nettprice is not null
df1.loc[df1['nettPrice'].notnull(),'price'] = df1.loc[df1['nettPrice'].notnull(),'nettPrice']

#remove nettprice and geo-coordinates
df1 = df1.drop(['nettPrice','x','y'],axis=1)

#convert area to sq feet, and type float
df1.area = df1.area.astype('float') * 10.76

#forcing to numeric before changing to int type
df1.price = pd.to_numeric(df1.price, errors='coerce').astype('int')

#create new column to hold calculated psf price
df1['Unit Price psf'] = (df1['price']/df1['area']).astype('int')

#parsing the contract date to proper pandas datetime format
df1.contractDate = pd.to_datetime(df1.contractDate, format='%m%y').dt.to_period('M')

In [25]:
#checking out the tenure column and find a way standardize data
print(Counter(df1.tenure))

Counter({'Freehold': 32006, '99 yrs lease commencing from 2018': 12172, '99 yrs lease commencing from 2014': 8865, '99 yrs lease commencing from 2015': 7423, '99 yrs lease commencing from 2016': 5990, '99 yrs lease commencing from 2013': 5341, '99 yrs lease commencing from 2017': 4789, '99 yrs lease commencing from 2011': 4007, '99 yrs lease commencing from 2012': 3697, '99 yrs lease commencing from 2010': 3214, '99 yrs lease commencing from 2019': 2432, '99 yrs lease commencing from 1997': 2306, '99 yrs lease commencing from 2008': 1743, '99 yrs lease commencing from 1996': 1499, '99 yrs lease commencing from 1995': 1263, '99 yrs lease commencing from 2000': 1162, '99 yrs lease commencing from 2007': 1118, '99 yrs lease commencing from 2006': 959, '99 yrs lease commencing from 1994': 925, '99 yrs lease commencing from 2002': 811, '99 yrs lease commencing from 2001': 794, '99 yrs lease commencing from 1993': 739, '99 yrs lease commencing from 2009': 722, '99 yrs lease commencing from 1

In [26]:
for i in range(df1.shape[0]):
    if (df1.loc[i,'tenure'] == 'NA'):
        #set data where tenure is NA to 999 years
        df1.loc[i,'lease left'] = 999
    elif (df1.loc[i,'tenure'][2] == ' ') & (df1.loc[i,'tenure'][-4:]=='hold'):
        #set data where tenure is 99 years but without start year to 99
        df1.loc[i,'lease left'] = 99
    elif (df1.loc[i,'tenure'][0] == '1'):
        df1.loc[i,'lease left'] = int(df1.loc[i,'tenure'][0:3]) - this_year + int(df1.loc[i,'tenure'][-4:])
    elif df1.loc[i,'tenure'][2] == ' ':
        df1.loc[i,'lease left'] = int(df1.loc[i,'tenure'][0:2]) - this_year + int(df1.loc[i,'tenure'][-4:])
    else:
        df1.loc[i,'lease left'] = 999

In [31]:
#checking the rest of the columns to make sure there is no unusual entries
print(Counter(df1.typeOfSale))
print(Counter(df1.propertyType))
print(Counter(df1.district))
print(Counter(df1.typeOfArea))

Counter({'3': 57168, '1': 54195, '2': 1669})
Counter({'Condominium': 51076, 'Apartment': 37418, 'Executive Condominium': 13096, 'Terrace': 5419, 'Semi-detached': 3097, 'Detached': 1408, 'Strata Terrace': 1187, 'Strata Semi-detached': 254, 'Strata Detached': 77})
Counter({'19': 17394, '15': 8012, '18': 6937, '23': 6884, '05': 6845, '03': 6584, '14': 6490, '10': 6105, '27': 5800, '09': 5159, '16': 4334, '13': 4065, '20': 3918, '21': 3646, '11': 2961, '28': 2960, '12': 2722, '22': 2553, '17': 1677, '25': 1602, '04': 1338, '08': 1244, '01': 1037, '02': 986, '07': 899, '26': 875, '06': 5})
Counter({'Strata': 103089, 'Land': 9943})


In [27]:
#Splitting into 2 dataframes: landed transactions and non-landed property transactions

df_non_landed = df1[df1.propertyType.isin(['Condominium','Apartment','Executive Condominium'])]
df_landed = df1[df1.propertyType.isin(['Terrace','Semi-detached','Detached','Strata Terrace',
                                       'Strata Semi-detached','Strata Detached'])]

In [ ]:
#finding out the en-bloc sales data
display(df_non_landed[df_non_landed.noOfUnits>1].sort_values('noOfUnits'))

#further splitting the en-bloc non-landed transactions from the non-landed dataframe
#enbloc deals are picked up by dual conditions of large deal value of more than $20mil and >1 unit per transaction.
#this is not a perfect method but is able to pick up all enbloc deals while minimizing non-enbloc transactions

df_enbloc = df_non_landed[(df_non_landed.noOfUnits>1) & (df_non_landed.price>20000000)]
df_non_landed = df_non_landed[(df_non_landed.noOfUnits<=10) & (df_non_landed.price<=20000000)]

In [43]:
#checking for errors in floorRange column
print(df_non_landed.floorRange.value_counts())

#amend the floorRange error in original URA data 
df_non_landed.loc[df_non_landed.floorRange=='B1-B5','floorRange'] = '01-05'
df_non_landed.loc[df_non_landed.floorRange=='-','floorRange'] = '01-05'

01-05    36377
06-10    26890
11-15    19277
16-20     8992
21-25     4161
26-30     2619
31-35     1827
36-40      875
41-45      307
46-50      116
51-55       62
56-60       36
61-65       26
66-70       21
71-75        4
Name: floorRange, dtype: int64


In [40]:
#remove floorRange column of landed because it is not relevant
df_landed = df_landed.drop(['floorRange'],axis=1)

01-05    36377
06-10    26890
11-15    19277
16-20     8992
21-25     4161
26-30     2619
31-35     1827
36-40      875
41-45      307
46-50      116
51-55       62
56-60       36
61-65       26
66-70       21
71-75        4
Name: floorRange, dtype: int64


In [29]:
#convert data types of various columns
df_non_landed = df_non_landed.astype({'area':'int32','noOfUnits':'int32','floorRange':'category',
                                      'typeOfSale':'category','propertyType':'category',
                                      'district':'category','typeOfArea':'category','tenure':'str'})

df_enbloc = df_enbloc.astype({'area':'int32','noOfUnits':'int32','floorRange':'category',
                              'typeOfSale':'category','propertyType':'category','district':'category',
                              'typeOfArea':'category','tenure':'str'})

df_landed = df_landed.astype({'area':'int32','noOfUnits':'int32','typeOfSale':'category','propertyType':'category',
                              'district':'category','typeOfArea':'category','tenure':'str'})

In [36]:
df_non_landed.groupby('floorRange')['Unit Price psf'].mean()

floorRange
01-05    1293.266707
06-10    1341.187096
11-15    1370.885356
16-20    1554.894684
21-25    1735.936073
26-30    1849.214968
31-35    1835.992885
36-40    1860.861714
41-45    2174.162866
46-50    2413.405172
51-55    2643.419355
56-60    2567.555556
61-65    2352.000000
66-70    2348.047619
71-75    2309.000000
Name: Unit Price psf, dtype: float64

In [47]:
x = df_non_landed.groupby('floorRange')['Unit Price psf'].mean()
x.pct_change().plot(kind='bar',ylabel='Percent change from lower floor range')

In [56]:
diff = df_non_landed.loc[df_non_landed.propertyType=='Apartment','Unit Price psf'].mean() - df_non_landed.loc[df_non_landed.propertyType=='Condominium','Unit Price psf'].mean()
print(diff)

In [59]:
df_non_landed.groupby('district')['Unit Price psf'].mean().sort_values()

district
25     786.349673
27     912.172255
17     974.659516
23     978.280261
26    1035.436229
22    1075.284940
18    1097.566797
28    1133.216244
19    1157.076194
16    1172.831649
05    1383.819797
12    1396.818182
21    1400.357547
20    1420.701415
14    1442.369620
08    1446.516762
15    1477.045901
04    1509.507886
13    1569.891707
11    1719.595878
03    1790.294618
06    1930.000000
10    2052.479745
01    2169.001929
02    2186.503568
09    2224.771484
07    2253.626251
Name: Unit Price psf, dtype: float64

In [60]:
df_landed.groupby('district')['Unit Price psf'].mean().sort_values()

district
25     747.930556
27     780.706989
22     784.205128
17     876.737069
18     900.655172
16    1001.588978
12    1050.469388
23    1073.200000
19    1106.496013
14    1148.210019
05    1148.817829
28    1174.045764
26    1200.856287
20    1219.144144
13    1226.446043
21    1329.010730
03    1338.000000
15    1409.787485
08    1435.619048
11    1565.392318
10    1568.161781
04    1689.728571
02    2833.400000
09    3034.179487
Name: Unit Price psf, dtype: float64

In [32]:
#saving the cleaned data into csv
#df_non_landed.to_csv("Non-landed transactions downloaded and cleaned on year " + str(this_year) + " month " + str(this_month) + ".csv")
#df_landed.to_csv("landed transactions downloaded and cleaned on year " + str(this_year) + " month " + str(this_month) + ".csv")
#df_enbloc.to_csv("enbloc transactions downloaded and cleaned on year " + str(this_year) + " month " + str(this_month) + ".csv")

In [33]:
#Analysis of single condo project
condo = 'CARIBBEAN'
df_condo = df_non_landed[df_non_landed['project'].str.contains(condo)]
df_condo.groupby('floorRange')['Unit Price psf'].mean()